In [1]:
import h2o
import random
import mlflow
import mlflow.h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
h2o.init()

wine = h2o.import_file(path="wine-quality.csv")
r = wine['quality'].runif()
train = wine[r  < 0.7]
test  = wine[0.3 <= r]

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_171"; Java(TM) SE Runtime Environment (build 1.8.0_171-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/fp/w6fpkjc56mv6bss0ww5w9l9r0000gn/T/tmp6bcj4_14
  JVM stdout: /var/folders/fp/w6fpkjc56mv6bss0ww5w9l9r0000gn/T/tmp6bcj4_14/h2o_toon_started_from_python.out
  JVM stderr: /var/folders/fp/w6fpkjc56mv6bss0ww5w9l9r0000gn/T/tmp6bcj4_14/h2o_toon_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Brussels
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,1 month and 24 days
H2O cluster name:,H2O_from_python_toon_qdlz2d
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [2]:
def trainRandomForest(ntrees):
    with mlflow.start_run():
        rf = H2ORandomForestEstimator(ntrees=ntrees)
        train_cols = [n for n in wine.col_names if n != "quality"]
        rf.train(train_cols, "quality", training_frame=train, validation_frame=test)
        
        mlflow.log_param("ntrees", ntrees)
        
        mlflow.log_metric("rmse", rf.rmse())
        mlflow.log_metric("r2", rf.r2())
        mlflow.log_metric("mae", rf.mae())
        
        mlflow.h2o.log_model(rf, "model")

In [3]:
for ntrees in [10, 20, 50, 100, 200]:
    trainRandomForest(ntrees)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
